# Phase 3.2: Full-Book Relationship Extraction with Provenance

**Inputs:**
- `06_document_structure_layer1.json` — 33 chapters, 3,449 paragraphs with citations
- `checkpoints/04_final_vocabulary.json` — 80 curated terms
- `checkpoints/04b_normalization_map.json` — canonical form map + coreference patterns

**Process:**
1. Load spaCy + EntityRuler with 240 patterns
2. Loop through every paragraph in document order
3. Extract relationships using `extract_buddhist_relationships_v3()`
4. Normalize subjects/objects through canonical map
5. Flag unresolved coreferences
6. Attach citation provenance to every relationship
7. Save incrementally by chapter (fault tolerance)

**Output:** `checkpoints/07_semantic_relationships.json`

Every relationship in this file can be traced back to the exact paragraph it came from.

In [3]:
import json
import os
import re
from typing import Dict, List, Any, Optional
from collections import defaultdict, Counter
from datetime import datetime

import spacy
from spacy.language import Language

# ── Paths ──
STRUCTURE_FILE = '06_document_structure_layer1.json'
VOCAB_FILE = 'checkpoints/04_final_vocabulary.json'
NORM_MAP_FILE = 'checkpoints/04b_normalization_map.json'
OUTPUT_FILE = 'checkpoints/07_semantic_relationships.json'

# ── Load document structure ──
with open(STRUCTURE_FILE) as f:
    doc_structure = json.load(f)

print(f"✓ Document structure: {doc_structure['total_chapters']} chapters, "
      f"{doc_structure['total_paragraphs']} paragraphs")

# ── Load normalization map ──
with open(NORM_MAP_FILE) as f:
    norm_data = json.load(f)

canonical_map = norm_data['canonical_map']
DEMONSTRATIVE_PREFIXES = norm_data['coreference_patterns']['demonstrative_prefixes']
PRONOUN_SUBJECTS = set(norm_data['coreference_patterns']['pronoun_subjects'])

print(f"✓ Normalization map: {len(canonical_map)} surface forms")

# ── Load vocabulary ──
with open(VOCAB_FILE) as f:
    vocab = json.load(f)

print(f"✓ Vocabulary: {vocab['metadata']['total_terms']} terms")

✓ Document structure: 33 chapters, 3449 paragraphs
✓ Normalization map: 1502 surface forms
✓ Vocabulary: 80 terms


## Set Up spaCy Pipeline

Rebuild the EntityRuler from the vocabulary — same 240 patterns as Phase 2.

In [4]:
# ── Load spaCy ──
nlp = spacy.load("en_core_web_lg")
print(f"✓ spaCy loaded: {nlp.meta['name']} v{nlp.meta['version']}")

# ── Build EntityRuler patterns from vocabulary ──
def generate_case_variants(term):
    variants = set()
    variants.add(term.lower())
    variants.add(term.capitalize())
    variants.add(term.title())
    return list(variants)

patterns = []
for category, terms in vocab['data'].items():
    if category in ['nouns', 'adj_noun']:
        label = 'CONCEPT'
    elif category == 'verbs':
        label = 'RELATIONSHIP_VERB'
    elif category == 'adj_prep':
        label = 'RELATIONSHIP_ADJPREP'
    else:
        continue
    
    for term, count in terms:
        for variant in generate_case_variants(term):
            patterns.append({"label": label, "pattern": variant})

# Deduplicate
seen = set()
unique_patterns = []
for p in patterns:
    key = (p['label'], p['pattern'])
    if key not in seen:
        seen.add(key)
        unique_patterns.append(p)

# Add to pipeline
if "entity_ruler" in nlp.pipe_names:
    nlp.remove_pipe("entity_ruler")

ruler = nlp.add_pipe("entity_ruler", before="ner")
ruler.add_patterns(unique_patterns)

print(f"✓ EntityRuler: {len(unique_patterns)} patterns")
print(f"  Pipeline: {nlp.pipe_names}")

✓ spaCy loaded: core_web_lg v3.8.0
✓ EntityRuler: 210 patterns
  Pipeline: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'entity_ruler', 'ner']


## Extraction Function (v3) + Normalization + Coreference

This is `extract_buddhist_relationships_v3()` from Phase 2, wrapped with normalization and coreference flagging.

In [5]:
# ── Normalization helpers ──

def make_canonical(term):
    t = re.sub(r'^(the|a|an)\s+', '', term.strip(), flags=re.IGNORECASE)
    t = t.lower().strip().replace(' ', '_')
    t = re.sub(r'_+', '_', t)
    return t

def normalize_concept(surface_form):
    if surface_form in canonical_map:
        return canonical_map[surface_form]
    if surface_form.lower() in canonical_map:
        return canonical_map[surface_form.lower()]
    stripped = re.sub(r'^(the|a|an)\s+', '', surface_form, flags=re.IGNORECASE)
    if stripped in canonical_map:
        return canonical_map[stripped]
    if stripped.lower() in canonical_map:
        return canonical_map[stripped.lower()]
    return make_canonical(surface_form)

# Collect unique terms for coreference check
unique_terms_lower = set()
for category, terms in vocab['data'].items():
    for term, count in terms:
        unique_terms_lower.add(term.lower())

def is_unresolved_reference(text):
    t = text.strip().lower()
    if t in PRONOUN_SUBJECTS:
        return True
    for prefix in DEMONSTRATIVE_PREFIXES:
        if t.startswith(prefix):
            remainder = t[len(prefix):]
            if remainder not in unique_terms_lower:
                return True
    return False


# ── Core extraction function (v3) ──

def extract_buddhist_relationships_v3(doc) -> List[Dict]:
    """
    Extract semantic relationships from a spaCy Doc.
    
    Pattern Types:
    1. RELATIONSHIP_VERB — Entity-Relation-Entity via marked verbs
    2. RELATIONSHIP_ADJPREP — Entity-Relation-Entity via adj+prep patterns
    3. TANTRIC_INSTRUCTION — Verb-Object-Prepositional phrase
    4. PREPOSITION — Simple spatial/locational relationships
    5. CONJUNCTION — Coordinated entities
    """
    relationships = []
    
    entities = {ent.root: ent for ent in doc.ents}
    concepts = [ent for ent in doc.ents if ent.label_ == 'CONCEPT']
    
    # Pattern 1 & 2: RELATIONSHIP_VERB and RELATIONSHIP_ADJPREP
    for ent in doc.ents:
        if ent.label_ in ['RELATIONSHIP_VERB', 'RELATIONSHIP_ADJPREP']:
            subject_ent = None
            object_ent = None
            
            for token in reversed(doc[:ent.start]):
                if token in entities and entities[token].label_ == 'CONCEPT':
                    subject_ent = entities[token]
                    break
            
            for token in doc[ent.end:]:
                if token in entities and entities[token].label_ == 'CONCEPT':
                    object_ent = entities[token]
                    break
            
            if subject_ent and object_ent:
                relationships.append({
                    'subject': subject_ent.text,
                    'subject_type': subject_ent.label_,
                    'relation': ent.text,
                    'relation_type': ent.label_,
                    'object': object_ent.text,
                    'object_type': object_ent.label_
                })
    
    # Pattern 3: TANTRIC_INSTRUCTION
    for token in doc:
        if token.pos_ == 'VERB' and token.dep_ in ('ROOT', 'advcl', 'relcl', 'xcomp'):
            verb_obj = None
            prep_obj = None
            prep_text = None
            
            for child in token.children:
                if child.dep_ == 'dobj':
                    for concept in concepts:
                        if concept.start <= child.i < concept.end:
                            verb_obj = concept
                            break
                if child.dep_ == 'prep':
                    prep_text = child.text
                    for grandchild in child.children:
                        if grandchild.dep_ == 'pobj':
                            for concept in concepts:
                                if concept.start <= grandchild.i < concept.end:
                                    prep_obj = concept
                                    break
            
            if verb_obj and prep_obj and prep_text:
                relationships.append({
                    'subject': verb_obj.text,
                    'subject_type': verb_obj.label_,
                    'relation': prep_text,
                    'relation_type': 'TANTRIC_INSTRUCTION',
                    'object': prep_obj.text,
                    'object_type': prep_obj.label_,
                    'verb': token.text
                })
    
    # Pattern 4: PREPOSITION
    for token in doc:
        if token.dep_ == 'prep' and token.head.i in [ent.root.i for ent in concepts]:
            subject_ent = None
            object_ent = None
            
            for concept in concepts:
                if concept.root.i == token.head.i:
                    subject_ent = concept
                    break
            
            for child in token.children:
                if child.dep_ == 'pobj':
                    for concept in concepts:
                        if concept.start <= child.i < concept.end:
                            object_ent = concept
                            break
            
            if subject_ent and object_ent:
                relationships.append({
                    'subject': subject_ent.text,
                    'subject_type': subject_ent.label_,
                    'relation': token.text,
                    'relation_type': 'PREPOSITION',
                    'object': object_ent.text,
                    'object_type': object_ent.label_
                })
    
    # Pattern 5: CONJUNCTION
    for concept in concepts:
        root = concept.root
        if root.dep_ == 'conj':
            for other_concept in concepts:
                if other_concept.root == root.head:
                    conj_word = 'AND'
                    for child in root.children:
                        if child.dep_ == 'cc':
                            conj_word = child.text.upper()
                            break
                    
                    relationships.append({
                        'subject': other_concept.text,
                        'subject_type': other_concept.label_,
                        'relation': conj_word,
                        'relation_type': 'CONJUNCTION',
                        'object': concept.text,
                        'object_type': concept.label_
                    })
                    break
    
    return relationships


# ── Wrapper: extract + normalize + flag coreference + attach provenance ──

def extract_with_provenance(text, paragraph_id, citation, chapter_title, chapter_index):
    """
    Full extraction pipeline for one paragraph:
    1. Run spaCy NER + dependency parsing
    2. Extract relationships (v3)
    3. Normalize concepts through canonical map
    4. Flag unresolved coreferences
    5. Attach provenance metadata
    """
    doc = nlp(text)
    raw_rels = extract_buddhist_relationships_v3(doc)
    
    processed = []
    for rel in raw_rels:
        subj_normalized = normalize_concept(rel['subject'])
        obj_normalized = normalize_concept(rel['object'])
        
        subj_unresolved = is_unresolved_reference(rel['subject'])
        obj_unresolved = is_unresolved_reference(rel['object'])
        resolved = not (subj_unresolved or obj_unresolved)
        
        processed.append({
            'subject': subj_normalized,
            'subject_surface': rel['subject'],
            'relation': rel['relation'],
            'relation_type': rel['relation_type'],
            'object': obj_normalized,
            'object_surface': rel['object'],
            'verb': rel.get('verb'),
            'resolved': resolved,
            'source': {
                'paragraph_id': paragraph_id,
                'citation': citation,
                'chapter_title': chapter_title,
                'chapter_index': chapter_index,
            }
        })
    
    # Also collect entities found (for concept inventory)
    entities_found = [
        {'text': ent.text, 'label': ent.label_, 'canonical': normalize_concept(ent.text)}
        for ent in doc.ents if ent.label_ == 'CONCEPT'
    ]
    
    return processed, entities_found

print("✓ Extraction pipeline defined")
print("  extract_buddhist_relationships_v3() — 5 pattern types")
print("  extract_with_provenance() — normalize + flag + cite")

✓ Extraction pipeline defined
  extract_buddhist_relationships_v3() — 5 pattern types
  extract_with_provenance() — normalize + flag + cite


## Test: Sample Paragraph

Verify the full pipeline works on one paragraph before running the whole book.

In [6]:
# Find a substantial teaching paragraph for testing
test_para = None
test_chapter = None
for ch in doc_structure['chapters']:
    for p in ch['paragraphs']:
        if (p.get('structural_role') in ('BODY', 'BODY_FIRST') 
            and len(p['text']) > 200
            and ch['chapter_index'] >= 7):  # Teaching chapters
            test_para = p
            test_chapter = ch
            break
    if test_para:
        break

print(f"Test paragraph: {test_para['citation']}")
print(f"Chapter: \"{test_chapter['chapter_title']}\"")
print(f"Text: \"{test_para['text'][:200]}...\"")
print()

rels, ents = extract_with_provenance(
    test_para['text'],
    test_para['paragraph_id'],
    test_para['citation'],
    test_chapter['chapter_title'],
    test_chapter['chapter_index']
)

print(f"Entities found: {len(ents)}")
for e in ents[:5]:
    print(f"  \"{e['text']}\" → {e['canonical']} ({e['label']})")

print(f"\nRelationships found: {len(rels)}")
for r in rels:
    flag = "✓" if r['resolved'] else "⚠ UNRESOLVED"
    verb = f" [verb: {r['verb']}]" if r['verb'] else ""
    print(f"  {flag} {r['subject']} --[{r['relation'].upper()}]--> {r['object']} ({r['relation_type']}){verb}")
    print(f"        Source: {r['source']['citation']}")

Test paragraph: CLB.7.§1.p10
Chapter: "Introduction and Preliminaries"
Text: "Samsara is like a vast ocean, for just as an ocean gives rise to waves, so rebirth in samsara gives rise to suffering. At the moment we have aprecious human body, which is the best vessel for crossing..."

Entities found: 4
  "Samsara" → samsara (CONCEPT)
  "samsara" → samsara (CONCEPT)
  "samsara" → samsara (CONCEPT)
  "enlightenment" → enlightenment (CONCEPT)

Relationships found: 0


## Full Book Extraction

Process all 3,449 paragraphs. Only processes teaching content (BODY, BODY_FIRST, VERSE, QUOTE_PROSE, LIST_ITEM, INSTRUCTION, SADHANA_PROSE). Skips INDEX, GLOSSARY, TOC, FRONT_MATTER, BACK_MATTER, headings.

Saves progress after each chapter for fault tolerance.

In [7]:
TEACHING_ROLES = {
    'BODY', 'BODY_FIRST', 'VERSE', 'QUOTE_PROSE', 'LIST_ITEM',
    'INSTRUCTION', 'SADHANA_PROSE', 'MANTRA', 'CHART_TITLE'
}

all_relationships = []
all_concepts_seen = Counter()  # canonical_form → count
chapter_stats = []
skipped_roles = Counter()
total_processed = 0

print("=" * 70)
print("FULL BOOK EXTRACTION")
print("=" * 70)

for ch in doc_structure['chapters']:
    ch_rels = []
    ch_entities = 0
    ch_paras_processed = 0
    
    for para in ch['paragraphs']:
        role = para.get('structural_role', 'BODY')
        
        # Skip non-teaching content
        if role not in TEACHING_ROLES:
            skipped_roles[role] += 1
            continue
        
        # Skip very short paragraphs (likely fragments)
        if len(para['text']) < 20:
            continue
        
        rels, ents = extract_with_provenance(
            para['text'],
            para['paragraph_id'],
            para.get('citation', para['paragraph_id']),
            ch['chapter_title'],
            ch['chapter_index'],
        )
        
        ch_rels.extend(rels)
        ch_entities += len(ents)
        ch_paras_processed += 1
        
        for e in ents:
            all_concepts_seen[e['canonical']] += 1
    
    all_relationships.extend(ch_rels)
    total_processed += ch_paras_processed
    
    stats = {
        'chapter_index': ch['chapter_index'],
        'chapter_title': ch['chapter_title'],
        'paragraphs_processed': ch_paras_processed,
        'relationships_found': len(ch_rels),
        'entities_found': ch_entities,
    }
    chapter_stats.append(stats)
    
    if ch_rels:
        print(f"  Ch {ch['chapter_index']:2d}: {ch['chapter_title'][:45]:45s} "
              f"| {ch_paras_processed:3d} paras | {len(ch_rels):3d} rels | {ch_entities:3d} ents")

# Summary
resolved_count = sum(1 for r in all_relationships if r['resolved'])
unresolved_count = len(all_relationships) - resolved_count

print(f"\n{'=' * 70}")
print(f"EXTRACTION COMPLETE")
print(f"{'=' * 70}")
print(f"  Paragraphs processed: {total_processed}")
print(f"  Total relationships:  {len(all_relationships)}")
print(f"    Resolved:           {resolved_count} ({100*resolved_count/max(1,len(all_relationships)):.1f}%)")
print(f"    Unresolved:         {unresolved_count} ({100*unresolved_count/max(1,len(all_relationships)):.1f}%)")
print(f"  Unique concepts seen: {len(all_concepts_seen)}")
print(f"  Skipped roles:        {dict(skipped_roles)}")

FULL BOOK EXTRACTION
  Ch  6: Preface                                       |   6 paras |   2 rels |  19 ents
  Ch  7: Introduction and Preliminaries                |  97 paras |  26 rels | 192 ents
  Ch  8: Channels, Winds and Drops                     |  77 paras |  13 rels | 123 ents
  Ch  9: Inner Fire                                    | 135 paras |  42 rels | 388 ents
  Ch 10: Clear Light and the Four Joys                 | 249 paras | 181 rels | 723 ents
  Ch 11: The Nine Mixings and the Two Mudras           | 101 paras | 109 rels | 542 ents
  Ch 12: Introduction to the Nature of the Mind        |  73 paras |  42 rels | 273 ents
  Ch 13: Tranquil Abiding                              |  65 paras |  24 rels | 197 ents
  Ch 14: Meditation on Emptiness                       |  74 paras |  14 rels | 162 ents
  Ch 15: Illusory Body                                 |  64 paras |  72 rels | 360 ents
  Ch 16: Clear Light and Union                         |  33 paras |  35 rels | 213 ents


## Extraction Statistics

In [8]:
# Relationship type distribution
type_dist = Counter(r['relation_type'] for r in all_relationships)
print("Relationships by type:")
for rtype, count in type_dist.most_common():
    print(f"  {rtype:25s}: {count:4d}")

# Most connected concepts
print(f"\nTop 20 concepts by frequency:")
for concept, count in all_concepts_seen.most_common(20):
    print(f"  {concept:35s}: {count:4d}")

# Sample resolved relationships
print(f"\nSample resolved relationships:")
resolved = [r for r in all_relationships if r['resolved']]
for r in resolved[:10]:
    verb = f" [verb: {r['verb']}]" if r['verb'] else ""
    print(f"  {r['subject']} --[{r['relation'].upper()}]--> {r['object']} ({r['relation_type']}){verb}")
    print(f"    Source: {r['source']['citation']}")

# Sample unresolved relationships
if unresolved_count > 0:
    print(f"\nSample UNRESOLVED relationships (not for Neo4j):")
    unresolved = [r for r in all_relationships if not r['resolved']]
    for r in unresolved[:5]:
        print(f"  \"{r['subject_surface']}\" --[{r['relation']}]--> \"{r['object_surface']}\"")
        print(f"    Source: {r['source']['citation']}")

Relationships by type:
  PREPOSITION              :  311
  RELATIONSHIP_VERB        :  186
  CONJUNCTION              :  125
  RELATIONSHIP_ADJPREP     :   44
  TANTRIC_INSTRUCTION      :   17

Top 20 concepts by frequency:
  mind                               :  618
  meditation                         :  350
  clear_light                        :  244
  emptiness                          :  226
  illusory_body                      :  200
  central_channel                    :  197
  secret_mantra                      :  136
  mahamudra                          :  127
  sleep                              :  118
  inner_fire                         :  118
  death                              :  109
  union                              :  104
  heruka                             :   84
  wisdom                             :   83
  completion_stage                   :   66
  enlightenment                      :   56
  spontaneous_great_bliss            :   55
  black_near-attainment     

## Save to JSON

In [9]:
output = {
    'metadata': {
        'version': '07_semantic_relationships',
        'timestamp': datetime.now().isoformat(),
        'source_book': doc_structure['book_id'],
        'extraction_function': 'extract_buddhist_relationships_v3',
        'normalization_map': NORM_MAP_FILE,
        'total_relationships': len(all_relationships),
        'resolved_relationships': resolved_count,
        'unresolved_relationships': unresolved_count,
        'unique_concepts': len(all_concepts_seen),
        'paragraphs_processed': total_processed,
        'relationship_types': dict(type_dist.most_common()),
    },
    'concept_inventory': {
        concept: count for concept, count in all_concepts_seen.most_common()
    },
    'chapter_stats': chapter_stats,
    'relationships': all_relationships,
}

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

file_size = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
print(f"✓ Saved to {OUTPUT_FILE}")
print(f"  File size: {file_size:.1f} MB")
print(f"  {len(all_relationships)} relationships")
print(f"  {len(all_concepts_seen)} unique concepts")

✓ Saved to checkpoints/07_semantic_relationships.json
  File size: 0.3 MB
  683 relationships
  50 unique concepts


## 🚦 Validation Gate 3B: Extraction with Provenance

In [10]:
print("=" * 70)
print("🚦 VALIDATION GATE 3B: Extraction with Provenance")
print("=" * 70)

# Reload to verify
with open(OUTPUT_FILE) as f:
    saved = json.load(f)

checks = []

# Check 1: Relationships exist
checks.append(('Relationships > 100', 
               len(saved['relationships']) > 100,
               f"{len(saved['relationships'])} relationships"))

# Check 2: Provenance complete
sample = saved['relationships'][0]
has_provenance = (
    'source' in sample and
    'paragraph_id' in sample['source'] and
    'citation' in sample['source'] and
    'chapter_title' in sample['source']
)
checks.append(('Provenance fields complete', has_provenance, ''))

# Check 3: Normalization working (no raw surface forms as subjects)
has_underscores = any('_' in r['subject'] for r in saved['relationships'][:50])
checks.append(('Concepts normalized (canonical forms)', has_underscores, ''))

# Check 4: Coreference flagging working
has_resolved_field = all('resolved' in r for r in saved['relationships'])
checks.append(('All relationships have resolved flag', has_resolved_field, ''))

# Check 5: Both resolved and unresolved exist (sanity check)
resolved_exists = any(r['resolved'] for r in saved['relationships'])
unresolved_exists = any(not r['resolved'] for r in saved['relationships'])
checks.append(('Both resolved and unresolved found', 
               resolved_exists and unresolved_exists,
               f"{saved['metadata']['resolved_relationships']} resolved, "
               f"{saved['metadata']['unresolved_relationships']} unresolved"))

# Check 6: Can trace back to source text
sample_rel = saved['relationships'][0]
para_id = sample_rel['source']['paragraph_id']
found = False
for ch in doc_structure['chapters']:
    for para in ch['paragraphs']:
        if para['paragraph_id'] == para_id:
            found = True
            break
    if found:
        break
checks.append(('Provenance traces to source paragraph', found, 
               f"{para_id} found in structure"))

# Check 7: Concept inventory exists
checks.append(('Concept inventory populated', 
               len(saved.get('concept_inventory', {})) > 20,
               f"{len(saved.get('concept_inventory', {}))} concepts"))

all_pass = True
for desc, passed, detail in checks:
    status = "✓" if passed else "✗"
    if not passed:
        all_pass = False
    detail_str = f" ({detail})" if detail else ""
    print(f"  {status} {desc}{detail_str}")

if all_pass:
    print(f"\n  ✅ GATE 3B PASSED")
    print(f"  Layer 2 semantic data ready for Neo4j!")
    print(f"  Resolved relationships → graph nodes + edges")
    print(f"  Unresolved relationships → preserved for future coreference work")
else:
    print(f"\n  ⚠️  SOME CHECKS FAILED")

# Show me test
print(f"\n{'─' * 70}")
print(f"'Show Me' test: A relationship and its source paragraph")
print(f"{'─' * 70}")
r = [r for r in saved['relationships'] if r['resolved']][0]
print(f"  {r['subject']} --[{r['relation'].upper()}]--> {r['object']}")
print(f"  Type: {r['relation_type']}")
print(f"  Citation: {r['source']['citation']}")
print(f"  Chapter: \"{r['source']['chapter_title']}\"")

# Find and show the source paragraph text
for ch in doc_structure['chapters']:
    for para in ch['paragraphs']:
        if para['paragraph_id'] == r['source']['paragraph_id']:
            print(f"  Source text: \"{para['text'][:200]}...\"")
            break

🚦 VALIDATION GATE 3B: Extraction with Provenance
  ✓ Relationships > 100 (683 relationships)
  ✓ Provenance fields complete
  ✓ Concepts normalized (canonical forms)
  ✓ All relationships have resolved flag
  ✗ Both resolved and unresolved found (683 resolved, 0 unresolved)
  ✓ Provenance traces to source paragraph (clb_ch6_para2 found in structure)
  ✓ Concept inventory populated (50 concepts)

  ⚠️  SOME CHECKS FAILED

──────────────────────────────────────────────────────────────────────
'Show Me' test: A relationship and its source paragraph
──────────────────────────────────────────────────────────────────────
  union --[OF]--> mahamudra
  Type: PREPOSITION
  Citation: CLB.6.p2
  Chapter: "Preface"
  Source text: "To attain pure realizations ofMahamudra, it is not sufficient merely to read these instructions. First we must train in the stages of the path common to both Sutra and Tantra by relying upon texts suc..."
